In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dotenv import load_dotenv
load_dotenv();

In [3]:
import base64
import json
import logging
import os
import re
import smtplib
import sys
import time
import threading
import traceback
from copy import deepcopy
from functools import partial, wraps
from itertools import chain
from importlib import reload
from pathlib import Path
from uuid import uuid4
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart

import bhutils
import blpapi
import ipyvuetify as v
import ipywidgets as w
import numpy as np
import pandas as pd
import clarion
import QuantLib as ql
import dataframe_image as dfi
import schedule
import win32com.client as win32
from bqplot import ColorScale, LinearScale, OrdinalColorScale, OrdinalScale
from ipydatagrid import BarRenderer, DataGrid, Expr, TextRenderer, VegaExpr
from IPython.display import display, clear_output
from pandas.tseries.offsets import BDay
from plotly import graph_objects as go
from plotly.subplots import make_subplots
from blp.blp import BlpParser

from api.watchdog import specs
from api.watchdog.specs import get_trades, get_targets
from api.watchdog.pricer import Pricer
from api.history_plotter_v2 import ExpressionHandler
from api.email import smtp_server, to_email_test, to_email_all, df_to_html, from_email, to_email_ext
from gioutils.clarion_ext import Runner
from gioutils.tools_clarion import get_clarion_positions
from gioutils.gui.pandas import millifyp as millify
from gioutils import ezutils as ez
from gioutils.blpw import BlpParser, BlpQuery
from gioutils.clarion_ext import super_clarion
from gioutils.tools import parse_swap_periods
from gioutils.ezutils import (
    bh,
    bhs,
    get_id,
    get_price_info,
)

from gioutils.utils import (
    get_bday,
    get_next_n,
    parse_offset,
    today,
    get_generic_future, 
    get_bbg_fut_chain_ticker,
    get_specific_future,
    get_root,
    encode_plotly_fig_as_image_tag,
)
from gioutils.tools_clarion import get_history_plotter_book

pd.options.plotting.backend = "plotly"
pd.options.display.float_format = partial(millify, pct=None, decimal=4)

In [4]:
from api.jobs.risk import daily_var_email
from api.jobs.risk_intraday import var_reduction_email
from api.jobs.macd_watcher import macd_signals_email, make_plotly_fig
from api.jobs.positions import history_plotter_email

In [5]:
clarion.server_login()
bq = BlpQuery(timeout=50000, parser=BlpParser(raise_security_errors=False)).start()
# ba = BHSysApi(password="A&3jg5Aj")

2023-05-09 10:04:10,558 - Authenticating...
2023-05-09 10:04:11,322 - Successfully authenticated


In [6]:
super_clarion(clarion)
runner = Runner(clarion)

### Functions

In [7]:
def blended_zscore(series, series_aged, alpha=2/3):
    zscore = series.subtract(series.mean()).div(series.std())
    zscore_aged = series.subtract(series_aged.mean()).div(series_aged.std())
    return alpha * zscore + (1- alpha) * zscore_aged

In [8]:
def get_tables_and_figures(series, series_aged, periods):
    yrs = periods
    to_plot = pd.concat([series, series_aged], axis=1).ffill().dropna()
    to_plots = [to_plot, *[to_plot.loc[(to_plot.index[-1] - parse_offset(yr)):].dropna() for yr in yrs]]
    
    tables = [
        pd.concat(
            [
                to_plot.iloc[0].rename('first').to_frame().T,
                to_plot.iloc[-1].rename('last').to_frame().T,
                to_plot.describe(),
                to_plot.zscore().iloc[-1].rename('zscore').to_frame().T,
                to_plot.iloc[-1].to_frame().T.set_axis(['blended_zscore']).assign(
                    **{
                        to_plot.columns[0]: blended_zscore(to_plot.iloc[:, 0], to_plot.iloc[:, 1]).iloc[-1],
                        to_plot.columns[1]: blended_zscore(to_plot.iloc[:, 0], to_plot.iloc[:, 1]).iloc[-1]
                      }
                )
                
            ]
        ).T
    for to_plot in to_plots]
    
    plotly_kwargs = dict(
        width=1200,
        height=600, 
        template='plotly_white',
    )
    
    fig_all = to_plot.plot(**plotly_kwargs)
    figs = [to_plot.plot(**plotly_kwargs) for to_plot in to_plots]

    
    return tables, figs

### Expression
It's a dictionary where the key corresponds to the name of the time series, while the value can be one of:
- **dict, tuple, list**: in this case it behaves like the historical pricer and uses the underlying **irbt_remote**
- **str**: in this case uses the old engine as in **translation** and the instrument is defined as **[s usd.sofr 11mx2y]**. Currently only swaps as supported using this method

### Example
Create an instance of **ExpressionHandler** and *call get_data(expressions, \*\*kwargs)*
The instance has now two dataframe attributes:
- **expressions_data**: contains all the calculated expressions
- **components_data**: contains all the individual components needed to calculate the expressions 

In [9]:
bhs.static()

In [10]:
d = get_history_plotter_book(clarion, bq, delta_mul=-1, delta_tol=499, aged_bottom=True)
expressions = d[1]
clear_output()

In [24]:
]

boe_feb_may      +0.7*[s GBP.SONIA 16mx1y] +0.6*[s GBP.SONIA 4m...
cmsswaptions                            -1.2*[s USD.SOFR 11mx359m]
er__sfr_sprd     -2.9*[s EUR.3M 13mx3m] +5.3*[s EUR.3M 19mx3m] ...
eur_con_stee     +7.7*[s EUR.3M 1yx1y] -6.7*[s EUR.3M 2yx1y] -4...
eur_fly          +1.3*[s EUR.3M 1yx1y] +1.6*[s EUR.3M 28mx3m] -...
eur_open         +0.5*[BBG OE1 Comdty YLD_YTM_MID] +0.2*[s EUR....
flat                                       +1.6*[s USD.SOFR 4mx3m]
jpypayersfly                          -0.6*[s JPY.DYENCALM 9mx10y]
sfr_fly          -0.4*[s USD.SOFR 13mx3m] -11.6*[s USD.SOFR 19m...
usd_open_opt     -4.8*[s USD.SOFR 10mx359m] +2.2*[s USD.SOFR 13...
ptf              boe_feb_may + cmsswaptions + er__sfr_sprd + eu...
boe_feb_mayA     +0.7*[s GBP.SONIA 13mx1y] +0.6*[s GBP.SONIA 1m...
cmsswaptionsA                            -1.2*[s USD.SOFR 8mx359m]
er__sfr_sprdA    -2.9*[s EUR.3M 10mx3m] +5.3*[s EUR.3M 16mx3m] ...
eur_con_steeA    +7.7*[s EUR.3M 9mx1y] -6.7*[s EUR.3M 21mx1y] 

In [26]:
expressions['eur_open'] = '+0.5*[BBG OE1 Comdty YLD_YTM_MID] +0.2*[s EUR.3M 13mx3m] -0.9*[s EUR.3M 1mx3m] -3.3*[s EUR.3M 4mx3m] -0*[s EUR.ESTER 1m] +0*[s EUR.ESTER 1mx1m]'

In [27]:
expressions['eur_openA'] = '+0.5*[BBG GTDEM5Y Govt YLD_YTM_MID] +0.2*[s EUR.3M 10mx3m] -0.9*[s EUR.3M 1m] -3.3*[s EUR.3M 1mx3m] -0*[s EUR.ESTER 1m] +0*[s EUR.ESTER 1m]'

In [28]:
eh = ExpressionHandler()  # create the ExpressionHandler object
eh.get_data(expressions.to_dict(), bq=bq, history='25y'); # call get_data();

### EMAIL

In [34]:
expressions.to_clipboard()

In [29]:
yrs = ['5y', '2y', '1y']

In [30]:
hp_table = df_to_html(d[1].rename('expr').to_frame())

In [31]:
res = {'ptf': get_tables_and_figures(eh.expressions_data['ptf'], eh.expressions_data['ptfA'], periods=yrs)}
for series, series_aged in zip(
    eh.expressions_data.drop(['ptf', 'ptfA'], axis=1).iloc[:, :eh.expressions_data.drop(['ptf', 'ptfA'], axis=1).shape[1] // 2], 
    eh.expressions_data.drop(['ptf', 'ptfA'], axis=1).iloc[:, eh.expressions_data.drop(['ptf', 'ptfA'], axis=1).shape[1] // 2:],
):
    tables, figs = get_tables_and_figures(eh.expressions_data[series], eh.expressions_data[series_aged], periods=yrs)
    res[series] = tables, figs

In [32]:
periods = [f"{round((eh.expressions_data.index[-1] - eh.expressions_data.dropna().index[0]).days / 365.25)}y", *yrs]
html_body = f"<h1 style=\"font-family: Calibri;\">History Plotter Expressions</h1>"
html_body = html_body + hp_table 
html_body = html_body  + ('<br>' * 2) + f"<h1 style=\"font-family: Calibri;\">Charts (when line goes down you make money, y-scale 1mm)</h1>" 
for name, (tables, figs) in res.items():
    for table, fig, period in zip(tables, figs, periods):
        title = f"<h2 style=\"font-family: Calibri;\">{name.upper()} {period.upper()}</h2>"
        html_df = df_to_html(table.style.format(precision=2))
        img_tag = encode_plotly_fig_as_image_tag(fig)
        html_body = html_body + title + html_df + img_tag

In [35]:
to_email = to_email_all

In [36]:
message = MIMEMultipart("alternative")
message["Subject"] = f'PTF HISTORY in DELTA terms as of: {pd.Timestamp.now(): %d%b%y %H:%M}'
message["From"] = from_email
message["To"] = ','.join(to_email)

In [37]:
html_ = MIMEText(html_body, "html")
message.attach(html_)
smtp = smtplib.SMTP(smtp_server)

In [38]:
smtp.sendmail(from_email, to_email, message.as_string())

{}